In [16]:
import os 
from dotenv import load_dotenv

load_dotenv()


True

In [17]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

## Langsmith Tracking and Tracing
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2', 'true')

In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="o1-mini")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x10f57b140> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f2e1700> root_client=<openai.OpenAI object at 0x10f6cd970> root_async_client=<openai.AsyncOpenAI object at 0x10f57b290> model_name='o1-mini' temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [19]:
result = llm.invoke("What is the capital of France?")

In [20]:
print(result.content)

The capital of France is **Paris**.


In [21]:
from langchain_groq import ChatGroq
model=ChatGroq(model="qwen-qwq-32B")
model.invoke("What is the capital of France?")

AIMessage(content="\n<think>\nOkay, the user is asking for the capital of France. Let me think. I know that France is a country in Western Europe. The capital cities I remember... Paris comes to mind. Wait, is there any other possibility? Maybe Lyon? No, Lyon is a major city but not the capital. Let me verify. From what I've studied before, Paris has been the capital for a long time. It's known for the Eiffel Tower, Louvre Museum, and other landmarks. I don't think there's been any recent change. Some countries have capitals that are not the most populous cities, like Brazil's capital is Brasília, not Rio or São Paulo. But in France's case, Paris is both the largest city and the capital. I can't recall any historical changes to France's capital either. So I'm pretty confident the answer is Paris. Let me make sure there's no trick in the question. The user might be testing basic knowledge, so straightforward answer should be fine. Yeah, I'm pretty sure it's Paris.\n</think>\n\nThe capit

In [22]:
### Prompt Engineering

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert AI Engineer. Provide me an answer to the question."),
    ("user", "{input}"),
])

In [23]:
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me an answer to the question.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [24]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10f8da510>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10f8dbec0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
### chaining
chain=prompt|model
chain


ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me an answer to the question.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10f8da510>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10f8dbec0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [26]:
response=chain.invoke({"input": "What is the capital of France?"})
print(response.content)

The capital of France is **Paris**. 🇫🇷  



In [27]:
### Output Parser

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

response=chain.invoke({"input": "Can you tell me about Langsmith?"})
print(response)

As an AI expert, I can tell you about Langsmith! 

Langsmith is an open-source framework developed by Replicate, designed to simplify the development and deployment of large language models (LLMs). Think of it as a toolbox specifically built for working with LLMs like your friendly neighborhood me!

Here are some key features and benefits of Langsmith:

**1. Streamlined Development:**

* **Easy Model Access:** Langsmith makes it straightforward to connect with various LLMs, including those hosted on Replicate, Hugging Face, and your own local models.
* **Modular Building Blocks:** It provides reusable components and templates, allowing you to quickly assemble and experiment with different LLM applications.
* **Interactive Debugging:**  Langsmith offers interactive tools for debugging and fine-tuning your LLM workflows, making the development process more efficient.

**2. Enhanced Deployment:**

* **Serverless Deployment:**  You can easily deploy your LLM applications serverlessly, leve

In [28]:
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()
output_parser.get_format_instructions()

'Return a JSON object.'

In [29]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate

output_parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query \n {format_instruction}\n {query}\n",
    input_variables=["query"],
    partial_variables={"format_instruction": output_parser.get_format_instructions},
)

In [30]:
prompt


PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': <bound method JsonOutputParser.get_format_instructions of JsonOutputParser()>}, template='Answer the user query \n {format_instruction}\n {query}\n')

In [31]:
chain=prompt | model | output_parser
response=chain.invoke({"query": "Can you tell me about Langsmith?"})
print(response)

{'name': 'Langsmith', 'description': 'Langsmith is an open-source framework for building and deploying AI applications, specifically focusing on large language models (LLMs).', 'key_features': ['Modular and extensible design', 'Support for multiple LLM backends (e.g., Llama, GPT-Neo)', 'Tools for fine-tuning and customizing LLMs', 'Deployment options for cloud, on-premise, and edge environments', 'Integration with other AI tools and services'], 'website': 'https://github.com/langsmith-ai/langsmith', 'status': 'Active development'}


In [32]:
##output parser
#from langchain_core.output_parsers import XMLOutputParser
from langchain.output_parsers.xml import XMLOutputParser

# XML Output Parser
output_parser = XMLOutputParser()

# Prompt that instructs the model to return XML
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond in this XML format: <response><answer>Your answer here</answer></response>"),
    ("human", "{input}")
])

# Build the chain
chain = prompt | model

# Run the chain
#response = chain.invoke({"input": "What is LangChain?"})

raw_output =chain.invoke({"input": "What is LangChain?"})

# Print result
print(raw_output)



content='<response><answer>LangChain is an open-source framework that simplifies the development of applications powered by large language models (LLMs). It provides tools and abstractions for connecting LLMs with other data sources, managing their memory, and building complex conversational flows.</answer></response>\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 39, 'total_tokens': 98, 'completion_time': 0.107272727, 'prompt_time': 0.002394937, 'queue_time': 0.090622341, 'total_time': 0.109667664}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--adb0ebb2-4f05-4787-a29e-7c15ede6049e-0' usage_metadata={'input_tokens': 39, 'output_tokens': 59, 'total_tokens': 98}


In [33]:
## With Pydantic
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

model = ChatOpenAI(temperature=0.7)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two tired!'}

In [34]:
### Assignment --- Chat prompt template


from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# 1. Instantiate your LLM
model = ChatOpenAI(temperature=0.0)  # deterministic output for structured tasks

# 2. Define the Pydantic model for your product schema
class Product(BaseModel):
    name: str = Field(..., description="The product's official name")
    details: str = Field(..., description="A brief description of the product")
    price_usd: float = Field(..., description="The price of the product in US dollars")

# 3. Create a JsonOutputParser around that model
parser = JsonOutputParser(pydantic_object=Product)

# 4. Build a PromptTemplate that injects the parser's format instructions
prompt = PromptTemplate(
    template=(
        "You are a helpful assistant that, given a product, returns valid JSON "
        "conforming to the following schema:\n\n"
        "{format_instructions}\n\n"
        "Product: {product}\n"
    ),
    input_variables=["product"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 5. Compose your chain: prompt → model → parser
chain = prompt | model | parser

# 6. Invoke with a product query
result: Product = chain.invoke({"product": "iPhone 15 Pro Max"})
print(result)



{'name': 'iPhone 15 Pro Max', 'details': 'The latest flagship smartphone from Apple with advanced features and technology.', 'price_usd': 1299.99}
